In [1]:
!pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from models_DQ_showoff import *
from IA_Trafficlight import *

In [3]:
write = True
board = Board(width = 31, height = 31, spawn_rate=5, max_spawn_batch=6, write=write,targetSuccessfulTrips=200)

In [4]:
IA = TrafficLightIA(board,epsilon=-1)

In [5]:
if write:
    with open("data.json", "w") as file:
        file.write("{\"steps\":[")
state = IA.env.reset()
all = [get_grid(IA.env)]
state = torch.tensor(state).to(IA.device)
# We reshape the state to be a 1x(11x31x31) tensor
state = state.view(1, 11*31*31)
print("Running model...")
state = state.float()
score = 0
done = False
steps = 0
while not done:
    action,wasRandom = IA.getMoves(state, IA.get_epsilon(0))
    # Give time so the cars can move
    for _ in range(3): 
        next_state, reward, done, _ = IA.env.step(action)
        all.append(get_grid(IA.env))
    next_state = torch.tensor(next_state).to(IA.device)
    next_state = next_state.view(1,11*31*31)
    next_state = next_state.float()
    state = next_state
    if steps % 20 == 0:
      print(f'[{0}] [{steps}]',"Action: ", action)
    score += reward
    steps += 1
print(f'[{0}] [{steps}]',"Action: ", action, "| Reward: ", reward, "| Was random action:", wasRandom)
print("Successful Trips:", IA.env.successful_trips, "| Crashes:", IA.env.crashes)
print("State:",state)
if write:
    with open("data.json", "a") as file:
        file.write("{}]}")

Running model...
[0] [0] Action:  10
[0] [20] Action:  10
[0] [40] Action:  10
[0] [60] Action:  10
[0] [80] Action:  10
[0] [100] Action:  10
[0] [120] Action:  10
[0] [140] Action:  10
[0] [160] Action:  10
[0] [180] Action:  10
[0] [200] Action:  10
[0] [212] Action:  10 | Reward:  -374730.0 | Was random action: False
Successful Trips: 206 | Crashes: 22.0
State: tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')


In [6]:
df = pd.DataFrame({"Grid":all})

In [7]:
%%capture
paleta = np.array(
    [
    # 0 -> highway
    [30 , 30 , 30 ],  
    # 1 -> up traffic light, red
    [255, 0  , 0  ],
    # 2 -> down traffic light, green
    [0  , 255, 0  ],
    # 3 -> road, dark green
    [0  , 60, 0  ],
    # 4 -> car looking up, purple
    [255, 0  , 255],
    # 5 -> car looking down, orange
    [255, 165, 0  ],
    # 6 -> car looking left, pink
    [255, 192, 203],
    # 7 -> car looking right, cyan
    [0  , 255, 255],
    # 8 -> lane
    [255, 255, 255]
    ])
all_grid = df
pinta_me = lambda matriz: paleta[matriz.astype(int)]
fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(pinta_me(all_grid.iloc[0][0]))
def animate(i):
    patch.set_data(pinta_me(all_grid.iloc[i][0]))
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [8]:
# Success rate
if board.successful_trips:
    print("Success rate:", "{:.2f}".format(board.successful_trips / (board.successful_trips + board.crashes)*100)+"%")
    print(f"Crashes: {int(board.crashes)}")
    print(f"Successful trips: {board.successful_trips}")
    print(f"Time stuck: {board.time_stuck}")
else:
    print("No successful trips")

Success rate: 90.35%
Crashes: 22
Successful trips: 206
Time stuck: 37871


In [9]:
anim

In [10]:
writergif = animation.PillowWriter(fps=30)
anim.save('IA.gif',writer=writergif)